# This is a draft

In [1]:
import sys
import os
import requests
import json
import pandas as pd
import myvariant


In [2]:
# Input
#user_input = input('Please input file PATH. ')
CSV_input = '/Users/manolodominguez/Desktop/git-repos/STP_mini_projects/Igor-Manuel/csv_file.csv'

# input to pd data frame
df = pd.read_csv(CSV_input) 

# An extra copy will be neccesary for manipulation.
df_draft = pd.read_csv(CSV_input) 

In [3]:
##### This is just an example, we need to specify how many things 
##### are we go to check in the input.

# Find Variant ID (rs000000) in CSV
rs = df[df.apply(lambda x:x.str.contains("rs"))].dropna(how='all').dropna(axis=1, how='all')

# Find Variant ID (NM_0000000) in CSV
NM = df[df.apply(lambda x:x.str.contains("NM"))].dropna(how='all').dropna(axis=1, how='all')

# Find Variant ID (NP_0000000) in CSV
NP = df[df.apply(lambda x:x.str.contains("NP"))].dropna(how='all').dropna(axis=1, how='all')

# Find ClinVar field (RCV#) in CSV
RCV = df[df.apply(lambda x:x.str.contains("RCV"))].dropna(how='all').dropna(axis=1, how='all')

# Save the found data in their respective dictionary

if rs.empty == False:
    ind = rs.index.to_list()
    vals = list(rs.stack().values)
    row2rs = dict(zip(ind, vals))
    # To avoid repetitions, we should need to delete the row
    # where a rs has been found.
    print(row2rs)

if NM.empty == False:
    ind = NM.index.to_list()
    vals = list(NM.stack().values)
    row2NM = dict(zip(ind, vals))
    print(row2NM)
    
if NP.empty == False:
    ind = NP.index.to_list()
    vals = list(NP.stack().values)
    row2NP = dict(zip(ind, vals))
    print(row2NP)
    
if RCV.empty == False:
    ind = RCV.index.to_list()
    vals = list(RCV.stack().values)
    row2RCV = dict(zip(ind, vals))
    print(row2RCV)


#https://rest.ensembl.org/documentation/info/vep_hgvs_get

{11: 'rs268', 56: 'rs56116432', 57: 'rs334'}
{1: 'NPM1', 35: 'PIANP'}


In [4]:
# Ensemble REST API
server = "http://rest.ensembl.org"

#Empty dict
rs2features ={}

for index, rs  in row2rs.items(): 
    
    # Rows where substring 'rs' has been found need to be delete to avoid repetition
    # This will be done in df_draft
    df_draft = df_draft.drop(index)
    
    # Request each rs found
    ext = "/variation/human/{}?".format(rs)
    r = requests.get(server+ext, headers={ "Content-Type" : "application/json"})
    decoded = r.json()

    # Dict of dict
    rs2features[rs] = decoded
# We need to filter this data. This data can be used to fill the ultimate VCF
# Or can be used also to verify that the values of the CSV are ook
# For example gene name or coodinates
    
# This step made here with variant ID must be made with ... 

In [5]:
# Another API I have found
# https://docs.myvariant.info/en/latest/index.html#
# It can read our CSV requesting position ID.

In [6]:
mv = myvariant.MyVariantInfo()

In [7]:
mv.getvariant('chr17:g.7578532A>G', fields = 'clinvar')

{'_id': 'chr17:g.7578532A>G',
 '_version': 4,
 'clinvar': {'_license': 'http://bit.ly/2SQdcI0',
  'allele_id': 27396,
  'alt': 'G',
  'chrom': '17',
  'cytogenic': '17p13.1',
  'gene': {'id': '7157', 'symbol': 'TP53'},
  'hg19': {'end': 7578532, 'start': 7578532},
  'hg38': {'end': 7675214, 'start': 7675214},
  'hgvs': {'coding': ['LRG_321t1:c.398T>C',
    'LRG_321t2:c.398T>C',
    'LRG_321t3:c.398T>C',
    'LRG_321t4:c.398T>C',
    'LRG_321t5:c.2T>C',
    'LRG_321t6:c.2T>C',
    'LRG_321t7:c.2T>C',
    'LRG_321t8:c.281T>C',
    'NM_000546.5:c.398T>C',
    'NM_001126112.2:c.398T>C',
    'NM_001126113.2:c.398T>C',
    'NM_001126114.2:c.398T>C',
    'NM_001126115.1:c.2T>C',
    'NM_001126116.1:c.2T>C',
    'NM_001126117.1:c.2T>C',
    'NM_001126118.1:c.281T>C',
    'NM_001276695.2:c.281T>C',
    'NM_001276696.2:c.281T>C',
    'NM_001276697.2:c.-80T>C',
    'NM_001276698.2:c.-80T>C',
    'NM_001276699.2:c.-80T>C',
    'NM_001276760.2:c.281T>C',
    'NM_001276761.2:c.281T>C'],
   'genomic'